In [1]:
import pandas as pd
from json import loads, dumps


In [2]:
datas = ['darmstadt_unis', 'mpqa', 'opener_en']

df1 = [pd.read_json(f"../{i}/train_convert.json") for i in datas]
df = pd.concat(df1)
df.reset_index(drop=True)

,sent_id,raw_words,words,aspects,holder,opinions
0,Colorado_Technical_University_Online_2_06-11-2...,NA I am a current student of their MBA program...,"[NA, I, am, a, current, student, of, their, MB...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
1,Colorado_Technical_University_Online_2_06-11-2...,NA I have been in the IT industry for 20 years...,"[NA, I, have, been, in, the, IT, industry, for...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
2,Colorado_Technical_University_Online_2_06-11-2...,"NA I am currently a director , and do make a s...","[NA, I, am, currently, a, director, ,, and, do...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
3,Colorado_Technical_University_Online_2_06-11-2...,"NA I did my research , and am concerned about ...","[NA, I, did, my, research, ,, and, am, concern...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
4,Colorado_Technical_University_Online_2_06-11-2...,"NA Especially , since I am planning for a PhD ...","[NA, Especially, ,, since, I, am, planning, fo...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'}]"
...,...,...,...,...,...,...
6132,opener_en/kaf/hotel/english00011_0b57ac0ca8569...,NA Golf courses can be easily reached from the...,"[NA, Golf, courses, can, be, easily, reached, ...","[{'index': 0, 'from': 11, 'to': 13, 'term': 't...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 17, 'to': 18, 'term': 'o..."
6133,opener_en/kaf/hotel/english00011_0b57ac0ca8569...,"NA Hotel offers nice spa , large pool area and...","[NA, Hotel, offers, nice, spa, ,, large, pool,...","[{'index': 0, 'from': 1, 'to': 2, 'term': 'Hot...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 2, 'to': 3, 'term': 'off..."
6134,opener_en/kaf/hotel/english00011_0b57ac0ca8569...,NA Hospitality and services at the highest lev...,"[NA, Hospitality, and, services, at, the, high...","[{'index': 0, 'from': 1, 'to': 2, 'term': 'Hos...","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 4, 'to': 8, 'term': 'at ..."
6135,opener_en/kaf/hotel/english00011_0b57ac0ca8569...,NA I will definitely come back !,"[NA, I, will, definitely, come, back, !]","[{'index': 0, 'from': 0, 'to': 1, 'term': 'NA'...","[{'index': 0, 'from': 1, 'to': 2, 'term': 'I'}]","[{'index': 0, 'from': 2, 'to': 7, 'term': 'wil..."


In [3]:
df.to_json(r'ddx.json', orient='records')